In [ ]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings

## 1. Downloading dataset and organising data

In [ ]:
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500.head()

In [ ]:
sp500['Symbol'].value_counts()

In [ ]:
sp500['Symbol'] = sp500['Symbol'].str.replace('.','-')
symbols_list = sp500['Symbol'].unique().tolist()

In [ ]:
end_date = '2023-09-27'

start_date = pd.to_datetime(end_date) - pd.DateOffset(365*8)

data = yf.download(tickers=symbols_list,start=start_date,end=end_date).stack()
data.index.names = ['date','ticker']
data.columns = data.columns.str.lower()
data

In [ ]:
data.describe()

In [ ]:
data.isnull().sum()

## 2. Garmman Klass Volatility

In [ ]:
data['german_klass_vol'] = ((np.log(data['high'])-np.log(data['low']))**2)/2-(2*np.log(2)-1)*((np.log(data['adj close'])-np.log(data['open']))**2)

data['rsi'] = data.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x,length=20))

data['bb_low'] = data.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x),length=20).iloc[:,0])

data['bb_mid'] = data.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])
                                                          
data['bb_high'] = data.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

In [ ]:
def compute_atr(stock_data):
  atr = pandas_ta.atr(high=stock_data['high'],low=stock_data['low'],close=stock_data['close'],length=14)
  
  return atr.sub(atr.mean()).div(atr.std())

In [ ]:
data['atr'] = data.groupby(level=1,group_keys=False).apply(compute_atr)

In [ ]:
def compute_macd(close):
  macd = pandas_ta.macd(close=close,length=20).iloc[:,0]
  return macd.sub(macd.mean()).div(macd.std())

data['macd'] = data.groupby(level=1,group_keys=False)['adj close'].apply(compute_macd)

data['dollar_volume'] = (data['adj close'] * data['volume']) / 1e6

In [ ]:
data.iloc[900000:]

## 3. Aggregate to monthly level and filter top 150 most liquid stocks for each month.

* To reduce training time and experiment with features and strategies, we convert the business-daily data to month-end frequency.

In [ ]:
last_cols = [c for c in data.columns.unique(0) if c not in ['dollar_volume','volume','open','high','low','close']]

data = (pd.concat([data.unstack('ticker')['dollar_volume'].resample('M').mean().stack('ticker').to_frame('dollar_volume'),data.unstack()[last_cols].resample('M').last().stack('ticker')],axis=1)).dropna()
data

* Calculate 5-year rolling average of dollar volume for each stocks before filtering.

In [ ]:
data['dollar_volume'] = (data.loc[:,'dollar_volume'].unstack('ticker').rolling(5*12,min_periods=12).mean().stack())

data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

data

## 4. Calculate Monthly Returns for different time horizons as features.

* To capture time series dynamics that reflect, for example, momentum patterns, we compute historical returns using the method .pct_change(lag), that is, returns over various monthly periods as identified by lags.

In [ ]:
def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:

        df[f'return_{lag}m'] = (df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df
  

In [ ]:
# data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna
data = data.groupby(level=1,group_keys=False).apply(calculate_returns)

data